In [242]:
import db_utils
import pandas as pd
from sqlalchemy import text
import datetime as dt

In [246]:
# Reloads the module to reflect changes, useful when source file changes needs to be reflected here
# from importlib import reload
# reload(db_utils)

# Establish a connection to the postgres data warehouse
pg_engine = db_utils.connect_to_data_warehouse()
print('Connected to the data mart.')

Connected to the data mart.


## Read data mart table

In [247]:
with pg_engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM sales_order_item_flat"))
    sales_order_item_flat_df = pd.DataFrame(result.fetchall(), columns=result.keys())

In [248]:
sales_order_item_flat_df.head()

,item_id,order_id,order_number,order_created_at,order_total,total_qty_ordered,customer_id,customer_name,customer_gender,customer_email,product_id,product_sku,product_name,item_price,item_qty_order,item_unit_total
0,13,9,A1123000010463,2016-11-23 16:12:16,2705.00,1,14,Shantae Guseo,Male,debest@verizon.net,62231,210768997,Red Rockstud Sandals,2700.00,1,2700.00
1,886,463,440,2016-12-27 17:34:04,2745.00,1,24,Chung Bolger,Female,hyper@outlook.com,62231,210768997,Red Rockstud Sandals,2720.00,1,2720.00
2,788,414,BOU12319,2016-12-19 17:12:21,4020.00,2,5,Florence Alavi,Female,keutzer@me.com,62231,210768997,Red Rockstud Sandals,2720.00,1,2720.00
3,67,32,A1128000039472,2016-11-28 14:06:44,360.56,1,14,Shantae Guseo,Male,debest@verizon.net,62985,210924831,Cotton Poplin Blue Shirt,330.56,1,330.56
4,69,33,A1128000040759,2016-11-28 14:12:28,240.07,1,14,Shantae Guseo,Male,debest@verizon.net,62986,210924859,Circular Hole Light Pink Jaquard Top,235.07,1,235.07


In [166]:
sales_order_item_flat_df.dtypes

item_id                       int64
order_id                      int64
order_number                 object
order_created_at     datetime64[ns]
order_total                 float64
total_qty_ordered             int64
customer_id                   int64
customer_name                object
customer_gender              object
customer_email               object
product_id                    int64
product_sku                  object
product_name                 object
item_price                  float64
item_qty_order                int64
item_unit_total             float64
dtype: object

In [167]:
sales_order_item_flat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6903 entries, 0 to 6902
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   item_id            6903 non-null   int64         
 1   order_id           6903 non-null   int64         
 2   order_number       6903 non-null   object        
 3   order_created_at   6903 non-null   datetime64[ns]
 4   order_total        6903 non-null   float64       
 5   total_qty_ordered  6903 non-null   int64         
 6   customer_id        6903 non-null   int64         
 7   customer_name      6903 non-null   object        
 8   customer_gender    6903 non-null   object        
 9   customer_email     6903 non-null   object        
 10  product_id         6903 non-null   int64         
 11  product_sku        6903 non-null   object        
 12  product_name       6903 non-null   object        
 13  item_price         6903 non-null   float64       
 14  item_qty

In [168]:
sales_order_item_flat_df.describe()

,item_id,order_id,order_created_at,order_total,total_qty_ordered,customer_id,product_id,item_price,item_qty_order,item_unit_total
count,6903.000000,6903.00000,6903,6903.000000,6903.000000,6903.000000,6.903000e+03,6903.000000,6903.000000,6903.000000
mean,103719.874258,57175.59916,2018-08-17 19:16:20.786759424,6041.891611,2.453426,30.164421,1.308047e+07,2401.582450,1.132551,2606.890110
min,13.000000,9.00000,2016-11-23 16:12:16,6.750000,1.000000,4.000000,5.306700e+04,5.100000,1.000000,5.100000
25%,4539.500000,2772.50000,2017-08-17 11:27:05.500000,775.760000,1.000000,24.000000,7.709500e+04,285.710000,1.000000,325.000000
50%,10726.000000,6925.00000,2018-08-14 15:12:27,2455.000000,2.000000,24.000000,1.415770e+05,1250.000000,1.000000,1320.000000
75%,23510.500000,16273.50000,2019-05-23 12:37:05,5904.880000,3.000000,24.000000,2.158700e+05,2748.695000,1.000000,2823.045000
max,410544.000000,228620.00000,2021-08-29 13:40:53,473800.000000,100.000000,149.000000,1.613532e+08,67000.000000,100.000000,214285.700000
std,166813.767301,90841.28158,NaN,20646.356696,2.604934,15.985398,4.374896e+07,4669.602106,1.641682,6295.287063


# Sales Analytics

## Daily sales, orders and quantities sold

In [178]:
# Extract just the date from order_created_at
sales_order_item_flat_df["order_date"] = sales_order_item_flat_df["order_created_at"].dt.date

# Group by order_date and compute total sales, orders, and quantity
daily_summary = (
    sales_order_item_flat_df.groupby("order_date")
    .agg(
        total_sales=("item_unit_total", "sum"),
        total_orders=("order_id", "nunique"),
        total_quantity=("item_qty_order", "sum")
    )
    .reset_index()
    .sort_values(by="order_date")
)

daily_summary.head(10)

,order_date,total_sales,total_orders,total_quantity
0,2016-11-23,2700.00,1,1
1,2016-11-24,1530.00,1,1
2,2016-11-28,16341.53,7,8
3,2016-11-29,975.00,1,1
4,2016-12-05,22891.80,5,10
5,2016-12-06,73277.14,22,30
6,2016-12-07,19703.16,6,7
7,2016-12-08,29267.01,7,9
8,2016-12-09,39031.20,14,14
9,2016-12-10,79931.53,46,53


## Top products by total sales

In [179]:
# Group by product and compute total sales and quantity sold
product_sales_summary = (
    sales_order_item_flat_df
    .groupby(["product_id", "product_name"])
    .agg(
        total_sales=("item_unit_total", "sum"),
        total_quantity=("item_qty_order", "sum")
    )
    .reset_index()
    .sort_values(by="total_sales", ascending=False)
)

# Display top 10 best-selling products by total sales
product_sales_summary.head(10)

,product_id,product_name,total_sales,total_quantity
1037,83943,Rose-Gold & Malachite Serpenti Sautoir Necklace,493333.30,11
1974,174955,"Rose-Gold, Leather & Diamond Serpenti Watch",428571.40,10
1038,83956,"Rose-Gold, Black Ceramic & Diamond Serpenti Sp...",276619.05,7
691,64504,Black Tiger-Embelished Ruffled Dress,215200.00,5
1042,83962,"Rose-Gold, Stainless Steel & Diamond Serpenti ...",201000.00,3
1040,83960,"Rose-Gold, White Ceramic & Diamond Serpenti Sp...",197547.62,5
1045,84037,Multi-coloured Desert Quartz Kosmos Scarf,187500.00,125
686,64485,Black Tiger-Embelished Ruffled Dress,170900.00,4
692,64507,Black Tiger-Embelished Ruffled Dress,153000.00,4
1039,83959,Stainless Steel & Diamond Serpenti Tubogas Watch,136761.92,4


## Best selling products by quantity (Top 10)

In [182]:
# Group by product and sum the quantities sold
best_selling_by_qty = (
    sales_order_item_flat_df.groupby("product_name")["item_qty_order"]
    .sum()
    .reset_index()
    .sort_values(by="item_qty_order", ascending=False)
)

# Show top 10
best_selling_by_qty.head(10)

,product_name,item_qty_order
1649,LSA Red Wine Gls Clr | 210942128,263
2685,Set Of Twelve Tweezer Sharpeners,206
1828,MA Palm Mini Frame:G | 209630413,169
1386,Gold Nail Ear Jackets,146
2013,Multi-coloured Desert Quartz Kosmos Scarf,125
1647,LSA Mixer Tumbler Ba | 210942137,55
960,"Clearly Corrective Dark Spot Solution, 30ml",53
1805,"Line-Reducing Eye-Brightening Concentrate, 15ml",40
1571,"Irish Leather Eau de Parfum, 75ml",34
1896,"Midnight Recovery Botanical Cleansing Oil, 175ml",31


# Customer Analytics

## Top customers by total spend

In [183]:
sales_by_customer = (
    sales_order_item_flat_df.groupby(["customer_id", "customer_name"])["item_unit_total"]
    .sum()
    .reset_index(name="total_sales")
    .sort_values("total_sales", ascending=False)
)

sales_by_customer.head()

,customer_id,customer_name,total_sales
13,24,Chung Bolger,12419300.06
20,41,Rex Ebonie,3233424.18
7,16,Cheryl Mcginnis,552899.57
31,67,Evonne Elenora,488070.06
41,98,Sherryl Socolow,242905.36


## Customer Order Frequency

In [186]:
order_counts = (
    sales_order_item_flat_df.groupby(["customer_id", "customer_name"])["order_id"]
    .nunique()
    .reset_index()
    .rename(columns={"order_id": "number_of_orders"})
    .sort_values(by="number_of_orders", ascending=False)
)

order_counts.head(10)

,customer_id,customer_name,number_of_orders
13,24,Chung Bolger,3014
20,41,Rex Ebonie,719
7,16,Cheryl Mcginnis,221
31,67,Evonne Elenora,116
41,98,Sherryl Socolow,60
26,49,Kaylene Vena,53
37,79,Flor Carmelo,34
24,46,Janie Penney,31
25,48,Lina Dacia,30
52,148,Henrietta Jeramy,16


## Customer Recency
Find the most recent order date for each customer — useful for churn prediction

In [187]:
latest_purchase = (
    sales_order_item_flat_df
    .groupby("customer_id")["order_created_at"]
    .max()
    .reset_index(name="last_order_date")
)

# Optional: Add today's date for recency calculation
latest_purchase["days_since_last_order"] = (
    pd.Timestamp("today").normalize() - latest_purchase["last_order_date"]
).dt.days

latest_purchase.sort_values("days_since_last_order", ascending=True).head(10)

,customer_id,last_order_date,days_since_last_order
13,24,2021-08-29 13:40:53,1367
20,41,2021-08-25 14:04:08,1371
31,67,2021-08-22 14:30:12,1374
7,16,2021-07-28 13:24:56,1399
37,79,2021-05-04 11:10:52,1484
11,22,2021-04-05 10:32:43,1513
4,10,2020-04-08 17:44:50,1875
1,5,2019-11-07 15:33:41,2028
21,42,2019-07-28 15:15:56,2130
27,50,2019-05-19 12:05:03,2200


## Sales Trend per Customer Segment (e.g., Gender)
Are there differences in revenue between male and female customers?

In [189]:
gender_sales = (
    sales_order_item_flat_df
    .groupby("customer_gender")["order_total"]
    .sum()
    .reset_index()
    .sort_values(by="order_total", ascending=False)
)

gender_sales

,customer_gender,order_total
0,Female,39307688.09
1,Male,2399489.70


## Customer Lifetime Value (CLV)
Estimate total revenue per customer

In [190]:
clv = (
    sales_order_item_flat_df.groupby(["customer_id", "customer_name"])["item_unit_total"]
    .sum()
    .reset_index(name="customer_lifetime_value")
    .sort_values("customer_lifetime_value", ascending=False)
)
clv.head()

,customer_id,customer_name,customer_lifetime_value
13,24,Chung Bolger,12419300.06
20,41,Rex Ebonie,3233424.18
7,16,Cheryl Mcginnis,552899.57
31,67,Evonne Elenora,488070.06
41,98,Sherryl Socolow,242905.36


## Average Order Value (AOV) by Customer

In [192]:
aov = (
    sales_order_item_flat_df.groupby("customer_id")
    .agg(total_revenue=("item_unit_total", "sum"), order_count=("order_id", "nunique"))
    .assign(avg_order_value=lambda x: x.total_revenue / x.order_count)
    .reset_index()
    .sort_values("avg_order_value", ascending=False)
)
aov.head()

,customer_id,total_revenue,order_count,avg_order_value
8,19,12600.00,1,12600.00
23,45,10500.00,1,10500.00
16,30,64050.00,7,9150.00
15,29,8550.00,1,8550.00
11,22,8428.58,1,8428.58


## RFM Analysis (Recency, Frequency, Monetary)
This is a classic marketing framework to score customers

In [196]:
snapshot_date = sales_order_item_flat_df["order_created_at"].max() + dt.timedelta(days=1)

rfm = (
    sales_order_item_flat_df.groupby("customer_id")
    .agg({
        "order_created_at": lambda x: (snapshot_date - x.max()).days,
        "order_id": "nunique",
        "item_unit_total": "sum"
    })
    .rename(columns={"order_created_at": "recency", "order_id": "frequency", "item_unit_total": "monetary"})
    .reset_index()
)

rfm.head()


,customer_id,recency,frequency,monetary
0,4,1477,14,34655.00
1,5,661,3,13991.43
2,6,1057,3,9444.75
3,7,1533,3,7000.00
4,10,508,7,11800.01


# Monthly Order Trends

In [197]:
monthly_trends = (
    sales_order_item_flat_df
    .groupby(sales_order_item_flat_df["order_created_at"].dt.to_period("M"))
    .agg(
        total_orders=("order_id", "nunique"),
        total_sales=("item_unit_total", "sum"),
        total_qty=("item_qty_order", "sum")
    )
    .reset_index()
    .rename(columns={"order_created_at": "month"})
)
monthly_trends["month"] = monthly_trends["month"].astype(str)  # Convert Period to string
monthly_trends.head()

,month,total_orders,total_sales,total_qty
0,2016-11,10,21546.53,11
1,2016-12,320,1071112.24,790
2,2017-01,99,278481.00,139
3,2017-02,99,382943.75,182
4,2017-03,234,398857.23,285


# Weekly Order Trends

In [198]:
weekly_trends = (
    sales_order_item_flat_df
    .groupby(sales_order_item_flat_df["order_created_at"].dt.to_period("W"))
    .agg(
        total_orders=("order_id", "nunique"),
        total_sales=("item_unit_total", "sum"),
        total_qty=("item_qty_order", "sum")
    )
    .reset_index()
    .rename(columns={"order_created_at": "week"})
)
weekly_trends["week"] = weekly_trends["week"].astype(str)  # Convert Period to string
weekly_trends.head()

,week,total_orders,total_sales,total_qty
0,2016-11-21/2016-11-27,2,4230.00,2
1,2016-11-28/2016-12-04,8,17316.53,9
2,2016-12-05/2016-12-11,121,338958.63,150
3,2016-12-12/2016-12-18,153,547363.61,577
4,2016-12-19/2016-12-25,9,19810.00,11


# Pricing Analytics

## Monthly Average Item Price Trend

In [199]:
monthly_avg_price = (
    sales_order_item_flat_df
    .groupby(sales_order_item_flat_df["order_created_at"].dt.to_period("M"))
    .agg(avg_price=("item_price", "mean"))
    .reset_index()
    .rename(columns={"order_created_at": "month"})
)

monthly_avg_price["month"] = monthly_avg_price["month"].astype(str)
monthly_avg_price.head()

,month,avg_price
0,2016-11,1958.775455
1,2016-12,2356.608646
2,2017-01,2181.387097
3,2017-02,2343.727124
4,2017-03,1477.452351


## Weekly Average Item Price Trend

In [200]:
weekly_avg_price = (
    sales_order_item_flat_df
    .groupby(sales_order_item_flat_df["order_created_at"].dt.to_period("W"))
    .agg(avg_price=("item_price", "mean"))
    .reset_index()
    .rename(columns={"order_created_at": "week"})
)

weekly_avg_price["week"] = weekly_avg_price["week"].astype(str)
weekly_avg_price.head()

,week,avg_price
0,2016-11-21/2016-11-27,2115.000000
1,2016-11-28/2016-12-04,1924.058889
2,2016-12-05/2016-12-11,2323.088944
3,2016-12-12/2016-12-18,2210.479676
4,2016-12-19/2016-12-25,1800.909091


# Product Performance Analytics

## Key Product Performance Metrics

In [201]:
product_performance = (
    sales_order_item_flat_df
    .groupby(["product_id", "product_name"])
    .agg(
        total_sales=("item_unit_total", "sum"),
        total_quantity_sold=("item_qty_order", "sum"),
        average_price=("item_price", "mean"),
        num_orders=("order_id", "nunique"),
        first_sold=("order_created_at", "min"),
        last_sold=("order_created_at", "max")
    )
    .reset_index()
    .sort_values(by="total_sales", ascending=False)
)

product_performance.head(10)

,product_id,product_name,total_sales,total_quantity_sold,average_price,num_orders,first_sold,last_sold
1037,83943,Rose-Gold & Malachite Serpenti Sautoir Necklace,493333.30,11,44867.722222,9,2017-08-08 13:18:59,2018-11-09 14:02:31
1974,174955,"Rose-Gold, Leather & Diamond Serpenti Watch",428571.40,10,42857.140000,4,2018-11-06 11:42:28,2018-11-08 17:03:57
1038,83956,"Rose-Gold, Black Ceramic & Diamond Serpenti Sp...",276619.05,7,39514.286000,5,2017-08-08 13:18:59,2018-11-09 14:22:06
691,64504,Black Tiger-Embelished Ruffled Dress,215200.00,5,42800.000000,3,2017-08-08 12:35:47,2017-08-15 12:57:28
1042,83962,"Rose-Gold, Stainless Steel & Diamond Serpenti ...",201000.00,3,67000.000000,3,2017-07-20 17:09:27,2017-08-13 11:15:32
1040,83960,"Rose-Gold, White Ceramic & Diamond Serpenti Sp...",197547.62,5,39511.905000,4,2017-08-17 12:55:53,2018-11-06 13:11:05
1045,84037,Multi-coloured Desert Quartz Kosmos Scarf,187500.00,125,1500.000000,100,2017-08-20 15:12:22,2017-11-21 09:59:23
686,64485,Black Tiger-Embelished Ruffled Dress,170900.00,4,42725.000000,4,2017-08-08 13:18:59,2017-09-26 10:37:30
692,64507,Black Tiger-Embelished Ruffled Dress,153000.00,4,38250.000000,2,2017-08-17 12:55:53,2017-08-17 12:57:57
1039,83959,Stainless Steel & Diamond Serpenti Tubogas Watch,136761.92,4,34190.480000,2,2018-11-06 13:33:57,2018-11-08 17:03:57


## Sales Consistency Score (Sales per Active Day)
This helps you spot products with steady performance vs. seasonal spikes.

In [202]:
product_performance["active_days"] = (
    (product_performance["last_sold"] - product_performance["first_sold"]).dt.days + 1
)
product_performance["sales_per_day"] = (
    product_performance["total_sales"] / product_performance["active_days"]
).round(2)

product_performance[["product_name", "sales_per_day"]].sort_values(by="sales_per_day", ascending=False).head(10)

,product_name,sales_per_day
692,Black Tiger-Embelished Ruffled Dress,153000.00
1974,"Rose-Gold, Leather & Diamond Serpenti Watch",142857.13
1041,"Rose-Gold, Stainless Steel & Diamond Serpenti ...",97800.00
1143,Yellow-Gold Serpent Boheme Clip Earrings,87600.00
1799,Pink Embellished Velvet Drape Gown,74057.16
958,Monchrome Gown With Gold Sequinned Underskirt,57700.00
499,SABINE GOWN - EXPOSED SHOULDERS LONG SLEEVE GO...,54165.00
445,Red Guipure Lace Dress,52500.00
2741,White-Gold & Diamond Pavé Pinky Ring,48190.48
310,3/4 SLEEVE GOWN:Ligh | 210912232,48000.00


## Average Order Value per Product

In [203]:
product_performance["avg_order_value"] = (
    product_performance["total_sales"] / product_performance["num_orders"]
).round(2)

product_performance[["product_name", "avg_order_value"]].sort_values(by="avg_order_value", ascending=False).head(10)

,product_name,avg_order_value
1974,"Rose-Gold, Leather & Diamond Serpenti Watch",107142.85
692,Black Tiger-Embelished Ruffled Dress,76500.00
691,Black Tiger-Embelished Ruffled Dress,71733.33
1039,Stainless Steel & Diamond Serpenti Tubogas Watch,68380.96
1042,"Rose-Gold, Stainless Steel & Diamond Serpenti ...",67000.00
958,Monchrome Gown With Gold Sequinned Underskirt,57700.00
1038,"Rose-Gold, Black Ceramic & Diamond Serpenti Sp...",55323.81
1037,Rose-Gold & Malachite Serpenti Sautoir Necklace,54814.81
445,Red Guipure Lace Dress,52500.00
1040,"Rose-Gold, White Ceramic & Diamond Serpenti Sp...",49386.90
